In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import transformers
import torch
import numpy as np

In [ ]:
df16k = pd.read_csv("/content/drive/MyDrive/NLPProj/EconomicURLs/output/out16k.csv")
df50k = pd.read_csv("/content/drive/MyDrive/NLPProj/EconomicURLs/output/out50k.csv")

In [ ]:
df16k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16708 entries, 0 to 16707
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0.2  16708 non-null  int64 
 1   Unnamed: 0.1  16708 non-null  int64 
 2   Unnamed: 0    16708 non-null  int64 
 3   url           16708 non-null  object
 4   emotion       16708 non-null  object
 5   headline      16708 non-null  object
 6   source        16708 non-null  object
 7   yr            16708 non-null  int64 
 8   sentiment     16708 non-null  int64 
 9   rel           16708 non-null  int64 
 10  short_title   12250 non-null  object
 11  full_title    12250 non-null  object
dtypes: int64(6), object(6)
memory usage: 1.5+ MB


In [ ]:
# initialize model
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("davidchiii/news-headlines")
model = AutoModelForSequenceClassification.from_pretrained("davidchiii/news-headlines", num_labels=3, problem_type="multi_label_classification")

In [ ]:
if torch.cuda.is_available(): 
 dev = "cuda:0" 
else: 
 dev = "cpu" 
device = torch.device(dev) 
print(device)

cpu


In [ ]:
# try it out
#input = tokenizer("DOJ watchdog reportedly sends criminal referral for McCabe to federal prosecutor", return_tensors='pt')
#input2 = tokenizer("Happy news. Child found after many days of smiling!", return_tensors='pt')

input = tokenizer("Houses are burning down because of global warming. No solution in sight.", return_tensors='pt')

with torch.no_grad():
    logits = model(**input).logits

print(logits)


tensor([[ 0.9117, -0.7314, -0.2652]])


In [ ]:
df16k['full_title']

0                                                      NaN
1        Mexican Gulf Seafood: Test Results Clean After...
2        Best Search Engine: Testing Blekko Vs. Google ...
3        Jobs Numbers, Home Prices, Stock Markets: Econ...
4        Richard Cordray: Consumer Financial-Protection...
                               ...                        
16703    Fed Plans Major Review of How it Pursues Infla...
16704    Atlanta Fed Leaves GDP View At 2.5 Percent | N...
16705    Global Nonfinancial Corporate Debt Hit Record ...
16706    Consumer Prices Unchanged as Underlying Inflat...
16707    Stocks Indexes Settle Down, but Small Companie...
Name: full_title, Length: 16708, dtype: object

In [ ]:
neg16k = np.zeros(len(df16k))
df16k['neg_full'] = neg16k
df16k['neu_full'] = neg16k
df16k['pos_full'] = neg16k

neg50k = np.zeros(len(df50k))
df50k['neg_full'] = neg50k
df50k['neu_full'] = neg50k
df50k['pos_full'] = neg50k

# add the columns

# print(df50k.head())

In [ ]:
for i in range(len(df16k['full_title'])):
  try:
    input = tokenizer(df16k['full_title'][i], return_tensors='pt')
    with torch.no_grad():
      logits = model(**input).logits
      df16k['neg_full'][i] = logits[0][0].item()
      df16k['neu_full'][i] = logits[0][1].item()
      df16k['pos_full'][i] = logits[0][2].item()

      print("16kvalue:", i)

      # print(df['neg'][i])
      # print(df['neu'][i])
      # print(df['pos'][i])
      # print(logits[0][0].item)
  except:
    pass

df16k.to_csv('16koutput.csv')

for i in range(len(df50k['full_title'])):
  try:
    input = tokenizer(df50k['full_title'][i], return_tensors='pt')
    with torch.no_grad():
      logits = model(**input).logits
      df50k['neg_full'][i] = logits[0][0].item()
      df50k['neu_full'][i] = logits[0][1].item()
      df50k['pos_full'][i] = logits[0][2].item()

      print("50kvalue:", i)

      # print(df['neg'][i])
      # print(df['neu'][i])
      # print(df['pos'][i])
      # print(logits[0][0].item)
    if i % 5000 == 0:
      df50k.to_csv('50koutput.csv')
  except:
    pass
  
df50k.to_csv('50koutput.csv')

<ipython-input-12-5b8cc24592e0>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df50k['neg_full'][i] = logits[0][0].item()
<ipython-input-12-5b8cc24592e0>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df50k['neu_full'][i] = logits[0][1].item()
<ipython-input-12-5b8cc24592e0>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df50k['pos_full'][i] = logits[0][2].item()


Streaming output truncated to the last 5000 lines.
50kvalue: 42739
50kvalue: 42740
50kvalue: 42741
50kvalue: 42742
50kvalue: 42743
50kvalue: 42744
50kvalue: 42745
50kvalue: 42746
50kvalue: 42747
50kvalue: 42748
50kvalue: 42749
50kvalue: 42750
50kvalue: 42751
50kvalue: 42752
50kvalue: 42753
50kvalue: 42754
50kvalue: 42755
50kvalue: 42756
50kvalue: 42757
50kvalue: 42758
50kvalue: 42759
50kvalue: 42760
50kvalue: 42761
50kvalue: 42762
50kvalue: 42763
50kvalue: 42764
50kvalue: 42765
50kvalue: 42766
50kvalue: 42767
50kvalue: 42768
50kvalue: 42769
50kvalue: 42770
50kvalue: 42771
50kvalue: 42772
50kvalue: 42773
50kvalue: 42774
50kvalue: 42775
50kvalue: 42776
50kvalue: 42777
50kvalue: 42778
50kvalue: 42779
50kvalue: 42780
50kvalue: 42781
50kvalue: 42782
50kvalue: 42783
50kvalue: 42784
50kvalue: 42785
50kvalue: 42786
50kvalue: 42787
50kvalue: 42788
50kvalue: 42789
50kvalue: 42790
50kvalue: 42791
50kvalue: 42792
50kvalue: 42793
50kvalue: 42794
50kvalue: 42795
50kvalue: 42796
50kvalue: 42797
50kva

In [ ]:
checkpoint16k = df16k
checkpoint50k = df50k


NameError: ignored